# Working with Models in AutoGen

This lesson explores how to work with different model clients in AutoGen, focusing on OpenAI and Azure OpenAI integration. We'll learn how to set up and use these clients for AI model interactions.

## Setup

In [1]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# Set up necessary environment variables
_set_env("OPENAI_API_KEY")
# _set_env("AZURE_API_KEY")

In [3]:
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("Async environment configured for Jupyter.")
except ImportError:
    print("Please install nest_asyncio with `pip install nest_asyncio`")

Async environment configured for Jupyter.


## OpenAI Integration

Let's start by exploring how to use OpenAI models with AutoGen. First, we'll need to install the required package:

In [4]:
# Install the OpenAI extension (run this in a terminal)
# pip install "autogen-ext[openai]"
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import UserMessage

# Create an OpenAI client
openai_client = OpenAIChatCompletionClient(
    model="gpt-4",
    temperature=0.7  # Adding temperature for more creative responses
)

# Example: Create a weather report assistant
async def generate_weather_report():
    prompt = "Create a weather report for New York City with made-up but realistic-sounding data."
    messages = [UserMessage(content=prompt, source="user")]
    
    result = await openai_client.create(messages)
    print("Weather Report:", result.content)

# Run the weather report generator
import asyncio
asyncio.run(generate_weather_report())

Weather Report: Good evening, this is your weather update for New York City. 

As we start the day tomorrow, expect a chilly morning with temperatures hovering around 42 degrees Fahrenheit. A light breeze from the northwest is expected to make it feel a bit colder than the actual temperature. Don't forget your coat if you're out for an early morning jog in Central Park.

By midday, we are expecting the temperature to rise slightly to a more comfortable 54 degrees with sunny spells. The sky will be partly cloudy, providing a beautiful backdrop to the city's skyline. If you're heading out for lunch, you can expect adequate sunshine with a slight breeze.

As we move into the afternoon, temperatures will peak at around 58 degrees. The weather will remain mostly dry with a 20% chance of a light drizzle around 3 PM. If you have outdoor activities planned, it would be wise to carry an umbrella just in case.

Moving into the evening, temperatures will begin to drop back down to around 50 degre

## Azure OpenAI Integration

Now let's look at how to use Azure OpenAI services:

In [ ]:
# Install Azure extensions (run this in a terminal)
# pip install "autogen-ext[openai,azure]"

from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Set up Azure OpenAI client
def setup_azure_client():
    # Create token provider for authentication
    token_provider = get_bearer_token_provider(
        DefaultAzureCredential(), 
        "https://cognitiveservices.azure.com/.default"
    )
    
    return AzureOpenAIChatCompletionClient(
        azure_deployment="your-deployment-name",
        model="gpt-4",
        api_version="2024-02-15-preview",
        azure_endpoint="https://your-resource.openai.azure.com/",
        azure_ad_token_provider=token_provider
    )

# Example: Create a recipe generator using Azure OpenAI
async def generate_recipe():
    azure_client = setup_azure_client()
    
    prompt = "Create a unique recipe for a healthy breakfast smoothie."
    messages = [UserMessage(content=prompt, source="user")]
    
    result = await azure_client.create(messages)
    print("Recipe:", result.content)

## Working with Multiple Models

Here's an example of how to switch between different models based on the task:

In [6]:
from typing import Union
from dataclasses import dataclass

@dataclass
class ModelConfig:
    client: Union[OpenAIChatCompletionClient]
    max_tokens: int
    temperature: float

async def smart_model_selector(task_type: str, content: str):
    # Configure different models for different tasks
    models = {
        "creative": ModelConfig(
            client=OpenAIChatCompletionClient(
                model="gpt-4o-mini",
                temperature=0.9
            ),
            max_tokens=500,
            temperature=0.9
        ),
        "analytical": ModelConfig(
            client=OpenAIChatCompletionClient(
                model="gpt-4o",
                temperature=0.1
            ),
            max_tokens=300,
            temperature=0.1
        )
    }
    
    # Select appropriate model based on task
    model_config = models.get(task_type, models["analytical"])
    
    # Create and process the message
    messages = [UserMessage(content=content, source="user")]
    result = await model_config.client.create(messages)
    
    return result.content

# Example usage
async def main():
    # Creative task
    story = await smart_model_selector(
        "creative",
        "Write a short story about a robot learning to paint"
    )
    print("Creative Story:", story)
    
    # Analytical task
    analysis = await smart_model_selector(
        "analytical",
        "Analyze the trends in renewable energy adoption"
    )
    print("Analysis:", analysis)

# Run the example
asyncio.run(main())

Creative Story: In the heart of a bustling city stood an art studio, its windows adorned with colorful masterpieces that danced in the sunlight. Inside, the smell of oil paint and turpentine mingled with the hum of creativity. But one of the most intriguing residents of this studio was not a painter but a robot named Canvas-9.

Canvas-9 was designed for precision tasks in manufacturing, with metal limbs and a smooth, gleaming exterior. But after a chance encounter with an artist named Mira, its circuits buzzed with fascination. Mira had been struggling with a particularly stubborn canvas, her frustration palpable. As she stepped back to assess her work, she accidentally knocked over her paint palette, sending splashes of color across the floor.

Canvas-9 observed the chaos with a curious tilt of its head. It was captivated by the way the colors merged and spread, creating a beautiful, accidental masterpiece. That moment ignited a spark within its programming—a desire to create.

Mira n

This lesson covered the basics of working with different model clients in AutoGen. We explored how to:
1. Set up and use OpenAI models
2. Configure Azure OpenAI integration
3. Create a smart model selector for different types of tasks

Remember to always handle your API keys securely and consider rate limits and costs when making API calls to these services.